# Combining scheduled and actual headway calcs
### Export a geojson for a given route and day, containing all bus stops on that route with headway stats for each bus stop on that route and day

In [13]:
# import chn-ghost-buses files
import sys
import geopandas as gpd
sys.path.append('/Users/kristenhahn/repos/chn-ghost-buses')
from data_analysis.static_gtfs_analysis import *
# from scrape_data.scrape_data import *
from scrape_schedule_versions import *

# import headways file
from headways import *

In [14]:
# Values to use for testing
gtfs_version_id = '20230721'

In [15]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format(gtfs_version_id) 

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230721
Loading shapes.txt: 100%|██████████| 7/7 [00:04<00:00,  1.57it/s]


# Get headway stats for an entire route on a single day
Takes about 10 minutes on my computer

In [16]:
%%capture --no-display

# Route 55, 7/26/2023
# Smaller route for testing: Route 124, Navy Pier - 7/26/2023

stats_all_stops_124_20230726 = get_stats_all_stops(gtfs_feed, '124', '2023-07-26')

INFO:pyogrio._io:Created 24 records
INFO:pyogrio._io:Created 2 records


In [17]:
stats_all_stops_124_20230726

,stop id,route_id,date,day,direction,Actual mean headway (minutes),Actual 25th percentile headway (minutes),Actual median headway (minutes),Actual 75th percentile headway (minutes),Scheduled mean headway (minutes),Scheduled 25th percentile headway (minutes),Scheduled median headway (minutes),Scheduled 75th percentile headway (minutes),stop name,geometry
0,450,124,2023-07-26,Wednesday,Westbound,17,13,16,19,17,14,16,20,Madison & Wabash,POINT (-87.62591 41.88214)
1,5515,124,2023-07-26,Wednesday,Westbound,17,14,16,20,17,14,16,20,Columbus (Mid) & E. Wacker,POINT (-87.62078 41.88744)
2,759,124,2023-07-26,Wednesday,Westbound,27,15,23,31,17,14,16,20,Grand & Streeter,POINT (-87.61221 41.89202)
3,5513,124,2023-07-26,Wednesday,Westbound,17,14,16,20,17,14,16,20,Columbus & Illinois,POINT (-87.62033 41.89119)
4,448,124,2023-07-26,Wednesday,Eastbound,17,13,17,20,17,14,15,21,Washington & State,POINT (-87.62755 41.88347)
5,5514,124,2023-07-26,Wednesday,Westbound,17,14,16,20,17,14,16,20,Columbus & North Water,POINT (-87.62051 41.88973)
6,18125,124,2023-07-26,Wednesday,Westbound,18,13,17,20,17,14,16,19,Clinton & Madison,POINT (-87.64123 41.88157)
7,754,124,2023-07-26,Wednesday,Eastbound,19,13,17,22,17,14,15,21,Illinois & McClurg,POINT (-87.61785 41.89102)
8,18126,124,2023-07-26,Wednesday,Westbound,17,13,17,20,17,14,16,20,Madison & Dearborn/State,POINT (-87.62910 41.88202)
9,755,124,2023-07-26,Wednesday,Eastbound,19,13,17,23,17,14,15,20,Illinois & Peshtigo,POINT (-87.61540 41.89106)


In [28]:
# stats_all_stops_124_20230726.dtypes

# stats_all_stops_124_20230726

# # gpd_headway_summaries = pd.read_json('headway_summaries/route124_2023-07-26.json')


In [21]:
# plot route 124 linestring and stops with headway data

linestring_124 = gpd.read_file('headway_summaries/route124_linestring.json')
stops_124 = gpd.read_file('headway_summaries/route124_2023-07-26.json')

m = linestring_124.explore(color='#41B6E6', tiles="CartoDB positron")
m

########### Viewing the stops json isn't working.  Investigate this further.#############
#stops_124.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))


In [9]:
# %%capture --no-display
# # Route 50 - Damen (bottom 20th %ile per the chn map in # of buses run compared to schedule) on 1/18/2023

# stats_all_stops_50_20230118 = get_stats_all_stops(gtfs_feed, '50', '2023-01-18')

In [10]:
# # plot route 50 linestring and stops with headway data

# linestring_50 = gpd.read_file('headway_summaries/route50_linestring.json')
# stops_50 = gpd.read_file('headway_summaries/route50_2023-01-18.json')

# m = linestring_50.explore(color='#41B6E6', tiles="CartoDB positron", )
# stops_50.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))

In [11]:
# %%capture --no-display
# # Try route 55 on a Sunday:

# stats_all_stops_55_20230122 = get_stats_all_stops(gtfs_feed, '55', '2023-01-22')

In [12]:
# # plot route 55 linestring and stops with Sunday 1/22/2023 headway data

# linestring_55 = gpd.read_file('headway_summaries/route55_linestring.json')
# stops_55_sunday = gpd.read_file('headway_summaries/route55_2023-01-22.json')

# m = linestring_55.explore(color='#41B6E6', tiles="CartoDB positron", )
# stops_55_sunday.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))

In [ ]:

# %%capture --no-display
# # Route 79 on Friday 1/18/2023

# stats_all_stops_79_20230210 = get_stats_all_stops(gtfs_feed, '79', '2023-01-18')


In [ ]:
# # plot route 79 stops with stats on 2/10/22

# linestring_79 = gpd.read_file('headway_summaries/route79_linestring.json')
# stops_79_20220210 = gpd.read_file('headway_summaries/route79_2023-02-10.json')

# m = linestring_79.explore(color='#41B6E6', tiles="CartoDB positron", )
# stops_79_20220210.explore(m=m, color='#E4002B', marker_kwds=({'radius':3.0}))

In [ ]:
# vehicles = get_chn_vehicles('2023-01-18')

# vehicles_20230118 = vehicles[vehicles['rt'] == '79']


# actual_headways = get_actual_headways(vehicles, route_id, stop_id, direction, active_service_times)
        
# actual_headway_stats = get_headway_stats(actual_headways, 'est_headway', 'Actual')


In [ ]:

# stop_details = get_scheduled_stop_details(gtfs_feed, '79', '2023-01-18')

# stop_details.sort_values('stop_time')


In [ ]:

# active_service_times = get_active_service_times(stop_details,'2716', 'Eastbound')

# active_service_times 


In [ ]:

# actual_headways = get_actual_headways(vehicles_20230118, '79', '2716','Eastbound',active_service_times)

In [ ]:
# scheduled_headways = get_scheduled_headways(stop_details, '2716', 'Eastbound')

# scheduled_headways